In [1]:
'''

6. Object detection using Transfer Learning of CNN architectures
a. Load in a pre-trained CNN model trained on a large dataset
b. Freeze parameters (weights) in model’s lower convolutional layers
c. Add custom classifier with several layers of trainable parameters to model
d. Train classifier layers on training data available for task
e. Fine-tune hyper parameters and unfreeze more layers as needed

dataset: caltech
'''


'\n\n6. Object detection using Transfer Learning of CNN architectures\na. Load in a pre-trained CNN model trained on a large dataset\nb. Freeze parameters (weights) in model’s lower convolutional layers\nc. Add custom classifier with several layers of trainable parameters to model\nd. Train classifier layers on training data available for task\ne. Fine-tune hyper parameters and unfreeze more layers as needed\n\ndataset: caltech\n'

In [2]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np


In [18]:

dataset_dir = "caltech-101-img/"
dataset_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
)

# here batch_size is the number of images in each batch
batch_size = 2000
dataset_generator = dataset_datagen.flow_from_directory(
    dataset_dir,
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 9144 images belonging to 102 classes.


In [19]:

x_train, y_train =  dataset_generator[0]
x_test, y_test = dataset_generator[1]

print(len(x_train))
print(len(x_test))

2000
2000


In [20]:

# Load VGG16 without top layers
weights_path = "vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
base_model = VGG16(weights=weights_path, include_top=False, input_shape=(64, 64, 3))

In [21]:
#b. Freeze parameters (weights) in model’s lower convolutional layers
for layer in base_model.layers:
   layer.trainable = False

In [22]:
#c. Add custom classifier with several layers of trainable parameters to model
x = Flatten()(base_model.output)
x = Dense(64, activation='relu')(x)
predictions = Dense(102, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)
# Compile the model
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
#d. Train classifier layers on training data available for task
# Train the model
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 245s 8s/step - accuracy: 0.8025 - loss: 0.9407 - val_accuracy: 0.5790 - val_loss: 1.8582
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 169s 5s/step - accuracy: 0.8165 - loss: 0.8550 - val_accuracy: 0.5830 - val_loss: 1.8203
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 113s 4s/step - accuracy: 0.8495 - loss: 0.7635 - val_accuracy: 0.5930 - val_loss: 1.8029
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 99s 3s/step - accuracy: 0.8675 - loss: 0.6888 - val_accuracy: 0.5975 - val_loss: 1.7965
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 174s 4s/step - accuracy: 0.8885 - loss: 0.6225 - val_accuracy: 0.6010 - val_loss: 1.7632
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 130s 4s/step - accuracy: 0.9045 - loss: 0.5609 - val_accuracy: 0.5950 - val_loss: 1.7726
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 97s 3s/step - accuracy: 0.9070 - loss: 0.5156 - val_accuracy: 0.5990 - val_loss: 1.7673
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 86s 3s/step - accuracy: 0.9305 - loss: 0.4613 - val_accuracy: 0.6080 - val_

In [25]:

import matplotlib.pyplot as plt
predicted_value = model.predict(x_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 44s 689ms/step


In [ ]:
labels = list(dataset_generator.class_indices.keys())

In [ ]:
n = 1000
plt.imshow(x_test[n])
print("Preditcted: ",labels[np.argmax(predicted_value[n])])
print("Actual: ", labels[np.argmax(y_test[n])])